In [17]:
import sys
import pathlib
from IPython.display import Audio

import librosa
import numpy as np
import pandas as pd
from langdetect import detect
from pythainlp.tokenize import word_tokenize
import deepcut
from torchmetrics.text import WordErrorRate, CharErrorRate
from string import punctuation
import re


ROOT_DIR = pathlib.Path.cwd().parent
sys.path.insert(0, str(ROOT_DIR))
from src.normalizers import EnglishTextNormalizer
from src.evaluate_utils import isd

In [18]:
target_folder = ROOT_DIR / 'data/result/Thonburian-whisper' / 'en_Digital_Roadshow_Q1_2023_NVD'
label_dir = target_folder / 'Groundtruth - Sheet2.csv'
th_result_dir = target_folder / 'th_result.csv'
en_result_dir = target_folder / 'en_result.csv'

In [19]:
label_df = pd.read_csv(str(label_dir))
th_result_df = pd.read_csv(str(th_result_dir)) 
en_result_df = pd.read_csv(str(en_result_dir))

In [20]:
merge_result_df = pd.concat(
    [th_result_df, en_result_df],
    axis=0
)
merge_result_df.shape

(707, 3)

In [21]:
sorted_result_df = merge_result_df.sort_values('start')

In [22]:
prediction = sorted_result_df['text'].to_list()
references = label_df['text'].to_list()

In [23]:
def tokenize_text(text: str) -> list:
    eng_normalizer = EnglishTextNormalizer()
    text = eng_normalizer(text)
    if not isinstance(text, str):
        return []
    try:
        lang = detect(text)
    except:
        lang = 'en'
    if lang != 'en':
        splited_text = deepcut.tokenize(text)
    else:
        splited_text = text.split(" ")
    return splited_text

In [24]:
prediction_words = list(map(tokenize_text, prediction))
references_words = list(map(tokenize_text, references))

1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 15ms/step


In [25]:
from functools import reduce

prediction_words_reduce = list(reduce(lambda a, b: a + b, prediction_words))
prediction_words_reduce = [word for word in prediction_words_reduce if not word in ["", " "]]
references_words_reduce = list(reduce(lambda a, b: a + b, references_words))
references_words_reduce = [word for word in references_words_reduce if not word in ["", " "]]

In [26]:
isd(prediction_words_reduce, references_words_reduce, debug=False) / len(references_words_reduce)

0.3214200619490112